In [1]:
import sys
sys.path.append('/Users/apple/Desktop/AiCore Bootcamp/multinational-retail-data-centralisation939')

In [2]:
from src.database_utils import DatabaseConnector
from src.data_cleaning import DataCleaning
from src.data_extraction import DataExtractor
from src.data_extraction import header


In [3]:
import numpy as np
import pandas as pd

## Task 3: Extract and clean user data

In [42]:
# Step 2: Read the credentials yaml file and return a dictionary of the credentials.

instance = DatabaseConnector('../db_creds.yaml')
db_creds = instance.read_db_creds()

In [43]:
# Step 3: Read the credentials from the return of read_db_creds and initialise and return an sqlalchemy database engine.
db_engine = instance.init_db_engine()

In [44]:
# Step 4: Create a method list_db_tables to list all the tables in the database so you know which tables you can extract data from.
table_names = instance.list_db_tables()
print(table_names)

['legacy_store_details', 'legacy_users', 'orders_table']


In [63]:
# Step 5:
# Develop a method called read_rds_table in your DataExtractor class which will extract the database table to a pandas DataFrame.

extractor = DataExtractor()
user_data = extractor.read_rds_table('legacy_users')
user_data.head()

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
index,,,,,,,,,,,
0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


In [64]:
missing_values_per_column = user_data.isnull().sum()

print("Number of missing values per column:")
print(missing_values_per_column)

Number of missing values per column:
first_name       0
last_name        0
date_of_birth    0
company          0
email_address    0
address          0
country          0
country_code     0
phone_number     0
join_date        0
user_uuid        0
dtype: int64


In [65]:
user_data.company.unique()

array(['Heydrich Junitz KG', 'Fox Ltd', 'Johnson, Jones and Harris', ...,
       'Robinson-Harris', 'Taylor, Fry and Jones',
       'Phillips, Brown and Powell'], dtype=object)

In [ ]:
df.loc[df['column_name'].apply(pd.to_numeric, errors='coerce').notna(), 'column_name'] = np.nan


In [81]:
def check_numeric_values(df, column_name):
      
        numeric_values = df[df[column_name].str.contains('\d', na=False)][column_name]

        # Print the numeric values
        print("Numeric values in 'column_name':")
        print(numeric_values.tolist())

check_numeric_values(user_data, 'country')

Numeric values in 'column_name':
['I7G4DMDZOZ', 'AJ1ENKS3QL', 'XGI7FM0VBJ', 'S0E37H52ON', 'XN9NGL5C0B', '50KUU3PQUF', 'EWE3U0DZIV', 'GMRBOMI0O1', 'YOTSVPRBQ7', '5EFAFD0JLI', 'PNRMPSYR1J', 'RQRB7RMTAD', '3518UD5CE8', '7ZNO5EBALT', 'T4WBZSW0XI']


In [86]:
def clean_strings(df, column_name):
        # Use a regular expression to identify rows with numeric characters
        numeric_rows = df[df[column_name].str.contains('\d', na=False)]

        # Remove rows with numeric values
        df = df[~df[column_name].str.contains('\d', na=False)]

        # Print the removed rows
        #print("Removed rows with numeric values:")
        #print(numeric_rows)        
        numeric_values = df[df[column_name].str.contains('\d', na=False)][column_name]

        # Print the numeric values
        print("Numeric values in 'column_name':")
        print(numeric_values.tolist())

clean_strings(user_data, 'country')

Numeric values in 'column_name':
[]


In [91]:
user_data = user_data[~user_data['first_name'].str.contains('\d', na=False)]
user_data['first_name'] = user_data['first_name'].replace('NULL', np.nan)

user_data.first_name.unique()

array(['Sigfried', 'Guy', 'Harry', ..., 'Jackson', 'Gertraude', 'Stephen'],
      dtype=object)

In [77]:
user_data.date_of_birth = user_data.date_of_birth.astype('string')
user_data.loc[user_data['join_date'].str.contains('[a-zA-Z]', na=False) & ~user_data['date_of_birth'].str.contains('-', na=False), 'date_of_birth'] = np.nan
        

In [55]:
user_data.shape[0]

15320

In [56]:
user_data = user_data.dropna().drop_duplicates()
user_data.shape[0]

15300

In [57]:
def missing(column_name):
    missing_values_count = user_data[column_name].isnull().sum()
    print(f"Number of missing values in 'column_name': {missing_values_count}")

In [62]:
missing('date_of_birth')

Number of missing values in 'column_name': 16


In [61]:
user_data.date_of_birth = user_data.date_of_birth.astype('string')
user_data.loc[user_data['join_date'].str.contains('[a-zA-Z]', na=False) & ~user_data['date_of_birth'].str.contains('-', na=False), 'date_of_birth'] = np.nan
user_data.shape[0]

15300

In [51]:
user_data.join_date = user_data.join_date.astype('string')
user_data.loc[user_data['join_date'].str.contains('[a-zA-Z]', na=False) & ~user_data['join_date'].str.contains('-', na=False), 'join_date'] = np.nan
user_data.shape[0]

15300

In [52]:
user_data.loc[:,'phone_number'] = user_data.phone_number.str.replace('(0)', '', regex=False)
user_data.phone_number = user_data.phone_number.apply(lambda x: ''.join(char for char in x if char.isdigit() or (char == '+' and x.startswith('+'))))
user_data.phone_number = user_data.phone_number.astype('string')
user_data.shape[0]

15300

In [8]:
# Step 6: Clean data
cleaner = DataCleaning()
try:
    cleaned_user_data = cleaner.clean_user_data(user_data)
except Exception as e:
    print(f"An error occurred: {e}")
    import traceback
    traceback.print_exc()

In [9]:
cleaned_user_data.head()

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
index,,,,,,,,,,,
0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,01614960674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+441214960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,03069990871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,01214960225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


In [10]:
# Step 7+8: Upload to SQL
instance = DatabaseConnector('../sales_data_creds.yaml')
instance.upload_to_db(cleaned_user_data, 'dim_users')

300

## Task 4: Extracting users and cleaning card details

In [11]:
# Step 2:
# Create a method in your DataExtractor class called retrieve_pdf_data, which takes in a link as an argument and returns a pandas DataFrame.

extractor = DataExtractor()
pdf_data = extractor.retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')

In [12]:
pdf_data.head()

,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13


In [13]:
#  Step 3: Clean data
cleaner = DataCleaning()
cleaned_card_data = cleaner.clean_card_data(pdf_data)

In [14]:
cleaned_card_data.head()

,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13


In [15]:
# Step 4: Upload to SQL
instance = DatabaseConnector('../sales_data_creds.yaml')
instance.upload_to_db(cleaned_card_data, 'dim_card_details')

299

## Task 5: Extract and clean details of the store

In [16]:
# Step 1+2:
# Create a method in your DataExtractor class called list_number_of_stores which returns the number of stores to extract. It should take in the number of stores endpoint and header dictionary as an argument.

number_of_stores_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'
extractor = DataExtractor()
number_of_stores = extractor.list_number_of_stores(number_of_stores_endpoint,header)

Number of stores: 451


In [17]:
# Step 3:
# Create another method retrieve_stores_data which will take the retrieve a store endpoint as an argument and extracts all the stores from the API saving them in a pandas DataFrame.
store_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'

store_data = extractor.retrieve_stores_data(store_endpoint, number_of_stores, header)

In [92]:
store_data.locality.unique()

array(['N/A', 'High Wycombe', 'Landshut', 'Westbury', 'Belper',
       'Gainsborough', 'Rutherglen', 'Stuttgart', 'Kaukauna', 'Hartley',
       'Siegburg', 'Devizes', 'Crystal Lake', 'Halstenbek', 'Lancing',
       'Newbury', 'Lymington', 'Chapletown', 'Barnet', 'East Dereham',
       'Poulton-le-Fylde', 'Bushey', 'Ehingen', 'Aberdeen', 'Surbiton',
       'Inverness', 'Weimar', 'Penzance', 'Albstadt', 'Hechingen',
       'Mexborough', 'Arbroath', 'Cowes', 'Clacton-on-Sea', 'Boston',
       'Charlottenburg-Nord', 'Bretten', 'Sun City Center', 'Karben',
       'Martinsburg', 'Oschatz', 'Kingston upon Hull', 'Leyland',
       'Venice', 'Schleswig', 'Eppingen', '9IBH8Y4Z0S', 'Losheim',
       'Burscheid', 'Southsea', 'Porterville', 'Walsrode', 'Exeter',
       'Thetford', 'Morningside Heights', 'Searcy', 'Radevormwald',
       'Stade', 'Mira Mesa', 'Strood', 'Verl', 'Wesseling', 'Pfullingen',
       'Bensheim', 'Walton-on-the-Naze', 'Friedberg', 'Winsford',
       'Newburgh', 'Selby', 'Obe

In [18]:
# Step 4: Clean data
cleaner = DataCleaning()
cleaned_store_data = cleaner.clean_store_data(store_data)

In [19]:
cleaned_store_data.head()

,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,NaN,<NA>,N/A,<NA>,325,2010-06-12,Web Portal,<NA>,GB,Europe
1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe


In [20]:
# Step 5: Upload to SQL
instance = DatabaseConnector('../sales_data_creds.yaml')
instance.upload_to_db(cleaned_store_data,'dim_store_details')

451

In [21]:
max_length_index = cleaned_store_data['continent'].str.len().idxmax()

# Retrieve the entry with the longest string length
entry_with_max_length = cleaned_store_data.loc[max_length_index, 'continent']

print("Entry with the longest length:", len(entry_with_max_length))

Entry with the longest length: 7


## Task 6: Extract and clean the product details

In [22]:
# Step 1: Extract data from s3 bucket into pd dataframe
extractor = DataExtractor()
products_data = extractor.extract_from_s3('s3://data-handling-public/products.csv')

In [23]:
products_data.head()

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6kg,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,Tiffany's World Day Out At The Park,£12.99,0.48kg,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,Tiffany's World Pups Picnic Playset,£7.00,590g,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,Tiffany's World Wildlife Park Adventures,£12.99,540g,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,Cosatto Cosy Dolls Pram,£30.00,1.91kg,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a


In [24]:
# Step 2: Convert product weights to correct units.
# Method is private as is used in the clean_products_data function.
# See data_cleaning.py convert_product_weights.

In [26]:
# Step 3: Clean whole dataframe
cleaner = DataCleaning()
cleaned_products_data = cleaner.clean_products_data(products_data)
cleaned_products_data.head()

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,still_available,R7-3126933h
1,Tiffany's World Day Out At The Park,£12.99,0.48,toys-and-games,<NA>,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,still_available,C2-7287916l
2,Tiffany's World Pups Picnic Playset,£7.00,0.59,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,still_available,S7-1175877v
3,Tiffany's World Wildlife Park Adventures,£12.99,0.54,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,Cosatto Cosy Dolls Pram,£30.00,1.91,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,still_available,B6-2596063a


In [27]:
cleaned_products_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1846 entries, 0 to 1852
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_name   1846 non-null   string
 1   product_price  1846 non-null   string
 2   weight         1846 non-null   object
 3   category       1846 non-null   string
 4   EAN            1668 non-null   string
 5   date_added     1844 non-null   string
 6   uuid           1846 non-null   string
 7   removed        1846 non-null   string
 8   product_code   1658 non-null   string
dtypes: object(1), string(8)
memory usage: 144.2+ KB


In [28]:
# Step 4: Upload to SQL
instance = DatabaseConnector('../sales_data_creds.yaml')
instance.upload_to_db(cleaned_products_data,'dim_products')

846

## Task 7: Retrieve and clean the orders table.

In [29]:
# Step 1: Find table containing information on product orders.
connector = DatabaseConnector('../db_creds.yaml')
connector.list_db_tables()

['legacy_store_details', 'legacy_users', 'orders_table']

In [30]:
# Step 2: Read orders_table file
extractor = DataExtractor()
orders_table_data = extractor.read_rds_table('orders_table')

In [31]:
orders_table_data.head()

,level_0,date_uuid,first_name,last_name,user_uuid,card_number,store_code,product_code,1,product_quantity
index,,,,,,,,,,
0,0,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,None,None,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8,30060773296197,BL-8387506C,R7-3126933h,NaN,3
1,1,0423a395-a04d-4e4a-bd0f-d237cbd5a295,None,None,8fe96c3a-d62d-4eb5-b313-cf12d9126a49,349624180933183,WEB-1388012W,C2-7287916l,NaN,2
2,2,65187294-bb16-4519-adc0-787bbe423970,None,None,fc461df4-b919-48b2-909e-55c95a03fe6b,3529023891650490,CH-01D85C8D,S7-1175877v,NaN,2
3,3,579e21f7-13cb-436b-83ad-33687a4eb337,None,None,6104719f-ef14-4b09-bf04-fb0c4620acb0,213142929492281,CL-C183BE4B,D8-8421505n,NaN,2
4,4,00ab86c3-2039-4674-b9c1-adbcbbf525bd,None,None,9523a6d3-b2dd-4670-a51a-36aebc89f579,502067329974,SO-B5B9CB3B,B6-2596063a,NaN,2


In [32]:
# Step 3: Clean orders_data
cleaned_orders_data = DataCleaning().clean_orders_data(orders_table_data)

In [33]:
cleaned_orders_data.head()

,date_uuid,user_uuid,card_number,store_code,product_code,product_quantity
index,,,,,,
0,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8,30060773296197,BL-8387506C,R7-3126933h,3
1,0423a395-a04d-4e4a-bd0f-d237cbd5a295,8fe96c3a-d62d-4eb5-b313-cf12d9126a49,349624180933183,WEB-1388012W,C2-7287916l,2
2,65187294-bb16-4519-adc0-787bbe423970,fc461df4-b919-48b2-909e-55c95a03fe6b,3529023891650490,CH-01D85C8D,S7-1175877v,2
3,579e21f7-13cb-436b-83ad-33687a4eb337,6104719f-ef14-4b09-bf04-fb0c4620acb0,213142929492281,CL-C183BE4B,D8-8421505n,2
4,00ab86c3-2039-4674-b9c1-adbcbbf525bd,9523a6d3-b2dd-4670-a51a-36aebc89f579,502067329974,SO-B5B9CB3B,B6-2596063a,2


In [34]:
cleaned_orders_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120123 entries, 0 to 118804
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   date_uuid         120123 non-null  string
 1   user_uuid         120123 non-null  string
 2   card_number       120123 non-null  string
 3   store_code        120123 non-null  string
 4   product_code      120123 non-null  string
 5   product_quantity  120123 non-null  int8  
dtypes: int8(1), string(5)
memory usage: 5.6 MB


In [35]:
# Step 4: Upload to SQL
connector = DatabaseConnector('../sales_data_creds.yaml')
connector.upload_to_db(cleaned_orders_data, 'orders_table')

123

## Task 8: Retrieve and clean the date events data.

In [36]:
# Step 1: Extract JSON file containing date events data
path = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json'

date_events_data = DataExtractor().extract_from_json(path)

In [37]:
date_events_data.head()

,timestamp,month,year,day,time_period,date_uuid
0,22:00:06,9,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad
1,22:44:06,2,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa
2,10:05:37,4,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3,17:29:27,11,2001,6,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44


In [38]:
# Step 2: Clean data
cleaned_date_events_data = DataCleaning().clean_date_data(date_events_data)

In [39]:
cleaned_date_events_data.head()

,timestamp,month,year,day,time_period,date_uuid
0,22:00:06,9,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad
1,22:44:06,2,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa
2,10:05:37,4,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3,17:29:27,11,2001,6,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44


In [40]:
# Step 3: Upload to SQL
connector = DatabaseConnector('../sales_data_creds.yaml')
connector.upload_to_db(cleaned_date_events_data, 'dim_date_times')

147

In [41]:
cleaned_date_events_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120147 entries, 0 to 120160
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   timestamp    120123 non-null  object
 1   month        120123 non-null  string
 2   year         120123 non-null  string
 3   day          120123 non-null  string
 4   time_period  120123 non-null  string
 5   date_uuid    120123 non-null  string
dtypes: object(1), string(5)
memory usage: 6.4+ MB
